<a href="https://colab.research.google.com/github/RenanBnu/A3_Inteligencia_Artificial/blob/main/anime_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Importando Bibliotecas**

Nova secção


In [ ]:
# Lendo o Dataset
import numpy as np
import pandas as pd
# Visualização
import plotly.express as px
import plotly.graph_objects as go  # for 3D plot visualization
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

from wordcloud import WordCloud
from langdetect import detect
from datetime import datetime

### **Carregando DataSet**

In [ ]:
# Configurando a exibição das colunas para 50
pd.set_option('display.max_columns', 50)

In [ ]:
# Importando arquivo csv dos dados dos animes
df_anime=pd.read_csv('/content/drive/MyDrive/anime-dataset/anime-dataset-2023.csv')
print("Tamanho do conjunto de dados:",df_anime.shape)
df_anime.head(3)

Tamanho do conjunto de dados: (24905, 24)


,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,Premiered,Status,Producers,Licensors,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",spring 1998,Finished Airing,Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",UNKNOWN,Finished Airing,"Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",spring 1998,Finished Airing,Victor Entertainment,"Funimation, Geneon Entertainment USA",Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...


In [ ]:
# Importando o DataFrame de Detalhes dos usuários
df_user=pd.read_csv('/content/drive/MyDrive/anime-dataset/users-details-2023.csv')
print("Tamanho do conjunto de dados:",df_user.shape)
df_user.head()

Tamanho do conjunto de dados: (731290, 16)


,Mal ID,Username,Gender,Birthday,Location,Joined,Days Watched,Mean Score,Watching,Completed,On Hold,Dropped,Plan to Watch,Total Entries,Rewatched,Episodes Watched
0,1,Xinil,Male,1985-03-04T00:00:00+00:00,California,2004-11-05T00:00:00+00:00,142.3,7.37,1.0,233.0,8.0,93.0,64.0,399.0,60.0,8458.0
1,3,Aokaado,Male,NaN,"Oslo, Norway",2004-11-11T00:00:00+00:00,68.6,7.34,23.0,137.0,99.0,44.0,40.0,343.0,15.0,4072.0
2,4,Crystal,Female,NaN,"Melbourne, Australia",2004-11-13T00:00:00+00:00,212.8,6.68,16.0,636.0,303.0,0.0,45.0,1000.0,10.0,12781.0
3,9,Arcane,NaN,NaN,NaN,2004-12-05T00:00:00+00:00,30.0,7.71,5.0,54.0,4.0,3.0,0.0,66.0,0.0,1817.0
4,18,Mad,NaN,NaN,NaN,2005-01-03T00:00:00+00:00,52.0,6.27,1.0,114.0,10.0,5.0,23.0,153.0,42.0,3038.0


In [ ]:
# Importing user score dataframe
df_score=pd.read_csv('/content/drive/MyDrive/anime-dataset/users-score-2023.csv')
print("Tamanho do conjunto de dados:",df_score.shape)
df_score.head()

Tamanho do conjunto de dados: (24325191, 5)


,user_id,Username,anime_id,Anime Title,rating
0,1,Xinil,21,One Piece,9
1,1,Xinil,48,.hack//Sign,7
2,1,Xinil,320,A Kite,5
3,1,Xinil,49,Aa! Megami-sama!,8
4,1,Xinil,304,Aa! Megami-sama! Movie,8


### **Análise Exploratória de Dados**

Para entender melhor os dados, examinaremos cada DataFrame usando o método info(), que oferece uma visão geral das colunas e da estrutura, além de ajudar a identificar valores ausentes.

In [24]:
# Verificando o dataframe df_anime
df_anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24905 entries, 0 to 24904
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   anime_id      24905 non-null  int64  
 1   Name          24905 non-null  object 
 2   English name  24905 non-null  object 
 3   Other name    24905 non-null  object 
 4   Score         24905 non-null  float64
 5   Genres        24905 non-null  object 
 6   Synopsis      24905 non-null  object 
 7   Type          24905 non-null  object 
 8   Episodes      24905 non-null  object 
 9   Aired         24905 non-null  object 
 10  Premiered     24905 non-null  object 
 11  Status        24905 non-null  object 
 12  Producers     24905 non-null  object 
 13  Licensors     24905 non-null  object 
 14  Studios       24905 non-null  object 
 15  Source        24905 non-null  object 
 16  Duration      24905 non-null  object 
 17  Rating        24905 non-null  object 
 18  Rank          20293 non-nu

In [19]:
# Verificando a coluna Score do df_anime
df_anime['Score'].value_counts()

,count
Score,
6.38,9282
6.31,80
6.54,80
6.51,79
6.25,79
...,...
3.21,1
3.29,1
1.85,1


In [18]:
# Filtra os valores da coluna 'Score', excluindo os que possuem o valor 'UNKNOWN'
scores = df_anime['Score'][df_anime['Score'] != 'UNKNOWN']

# Converte pra float
scores = scores.astype('float')

# Calcula a média dos valores numéricos na coluna 'Score', arredondando para 2 casas decimais
score_mean = round(scores.mean(), 2)

# Substitui todos os valores 'UNKNOWN' na coluna 'Score' pela média calculada
df_anime['Score'] = df_anime['Score'].replace('UNKNOWN', score_mean)

# Converte a coluna Score para float, permitindo calculos
df_anime['Score'] = df_anime['Score'].astype('float64')

In [22]:
# Processamento da coluna de Rank
df_anime['Rank'].value_counts()

,count
Rank,
0.0,187
12611.0,4
18804.0,4
18715.0,4
6577.0,4
...,...
18424.0,1
18423.0,1
11642.0,1


In [21]:
df_anime['Rank'] = df_anime['Rank'].replace('UNKNOWN', np.nan)
df_anime['Rank'] = df_anime['Rank'].astype('float64')

In [ ]:
# Verificando o dataframe df_user
df_user.info()

In [27]:
# Verificar quais colunas do dataframe df_score estão com valores nulos
df_score.isnull().sum()

,0
user_id,0
Username,232
anime_id,0
Anime Title,0
rating,0


In [29]:
df_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24325191 entries, 0 to 24325190
Data columns (total 5 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   user_id      int64 
 1   Username     object
 2   anime_id     int64 
 3   Anime Title  object
 4   rating       int64 
dtypes: int64(3), object(2)
memory usage: 927.9+ MB
